In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as th

In [2]:
tfds.list_builders()#imdb_reviews
dir(tfds)

['GenerateMode',
 'ReadConfig',
 'Split',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'absolute_import',
 'as_numpy',
 'audio',
 'builder',
 'core',
 'decode',
 'disable_progress_bar',
 'division',
 'download',
 'features',
 'file_adapter',
 'image',
 'is_dataset_on_gcs',
 'list_builders',
 'load',
 'object_detection',
 'percent',
 'print_function',
 'public_api',
 'show_examples',
 'structured',
 'summarization',
 'testing',
 'text',
 'tf_compat',
 'translate',
 'units',
 'version',
 'video']

In [3]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", #Used this dataset to train the model.
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXMREWR/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXMREWR/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXMREWR/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
list(train_data.batch(1))[0]

(<tf.Tensor: shape=(1,), dtype=string, numpy=
 array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."],
       dtype=object)>, <tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>)

In [0]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = th.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

In [6]:

model = tf.keras.models.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(tf.keras.layers.Dense(6,activation ="relu"))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 400,465
Trainable params: 400,465
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
hist = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 105ms/step - loss: 0.7005 - accuracy: 0.5077 - val_loss: 0.6870 - val_accuracy: 0.5027
Epoch 2/20
30/30 [==============================] - 3s 105ms/step - loss: 0.6795 - accuracy: 0.5279 - val_loss: 0.6702 - val_accuracy: 0.5920
Epoch 3/20
30/30 [==============================] - 3s 104ms/step - loss: 0.6607 - accuracy: 0.6349 - val_loss: 0.6526 - val_accuracy: 0.6740
Epoch 4/20
30/30 [==============================] - 3s 105ms/step - loss: 0.6442 - accuracy: 0.6922 - val_loss: 0.6384 - val_accuracy: 0.7031
Epoch 5/20
30/30 [==============================] - 3s 104ms/step - loss: 0.6301 - accuracy: 0.7247 - val_loss: 0.6269 - val_accuracy: 0.7447
Epoch 6/20
30/30 [==============================] - 3s 104ms/step - loss: 0.6182 - accuracy: 0.7586 - val_loss: 0.6160 - val_accuracy: 0.7583
Epoch 7/20
30/30 [==============================] - 3s 106ms/step - loss: 0.6064 - accuracy: 0.7856 - val_loss: 0.6087 - val_accuracy: 0.7855
Epoch 

In [9]:
model.predict(["He is a very good teacher"])#values near 1 are positive and near 0 are negative.

array([[0.99357176]], dtype=float32)

In [10]:
model.predict(["He is very bad in corrction of papers"])

array([[0.06287391]], dtype=float32)

In [11]:
model.predict(["good"])#most commonly used positive word

array([[0.6905314]], dtype=float32)

In [12]:
model.predict(["bad"])#most commonly used negative word

array([[0.00134106]], dtype=float32)

In [13]:
#I found a python library with  combination with NLTk library
#This just to ensure if the prediction is biased on not
#In this library, 1-purely +ve , -1-purely -ve , 0-neutral  
from textblob import TextBlob as tb
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
text = "He is a very good teacher"
blob =tb(text)
blob.sentences[0].sentiment.polarity

0.9099999999999999

In [15]:
text_2 ="He is worst in DSA"
blob_2 =tb(text_2)
blob_2.sentences[0].sentiment.polarity

-1.0

In [0]:
#i may combine these two to give sharper results.
#now i will add the real dataset, we need to check
#When more reviews come.

In [17]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
data=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/VIT-FACULTY REVIEW.csv')
data.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


,Which school does the teacher belongs to,NAME,REVIEWS
0,SCOPE,ANURADHA G,She is mediocre teacher and is very lenient in...
1,SITE,SANJIBAN SEKHAR ROY,Worst in teaching DSA
2,SMEC,RAHUL SINGH SIKARWAR,Worst teacher ever. Literally failed entire cl...
3,SCOPE,SATHIYA KUMAR C,bhen ki lodi hai
4,SCOPE,BALAKRISHNAN P,One of the most talented teacher in VIT. But c...


In [0]:
def content_check_1(x):#for textBlob(NLTK)
  if x < 0:
    return 1
  elif x > 0:
    return 3
  else:
    return 2 

def content_check_2(x):#for Tensorflow model
  if x < (1/3):
    return 1
  elif x > (1-1/3):
    return 3
  else:
    return 2


  

In [19]:
l=list()
for i in data["REVIEWS"]:
  text = i
  blob = tb(text)
  rating_1 = content_check_1(blob.sentences[0].sentiment.polarity)
  rating_2 = content_check_2(model.predict([text])[0][0])
  result = round((rating_1 + rating_2)/2) #combining two analysis to get a better result
  l.append(result)
data["RATINGS"] = l
data.head(10)

,Which school does the teacher belongs to,NAME,REVIEWS,RATINGS
0,SCOPE,ANURADHA G,She is mediocre teacher and is very lenient in...,3
1,SITE,SANJIBAN SEKHAR ROY,Worst in teaching DSA,2
2,SMEC,RAHUL SINGH SIKARWAR,Worst teacher ever. Literally failed entire cl...,1
3,SCOPE,SATHIYA KUMAR C,bhen ki lodi hai,2
4,SCOPE,BALAKRISHNAN P,One of the most talented teacher in VIT. But c...,3
5,SMEC,ANIL P.M.,Very chill in class,3
6,SMEC,ANOJ GIRI,Very chill in class. You can leave the class a...,3
7,SMEC,ARUNA KUMAR BEHURA,Okay-ish,2
8,SCOPE,NARESH K,One of the best teacher in VIT. He can be litt...,3
9,SCOPE,GAYATHRI P,Everyone said she is blacklisted but I think s...,2


In [0]:
data = data.sort_values(["Which school does the teacher belongs to","RATINGS"],axis="rows",ascending=False)

In [0]:
data["SCHOOL"] = data["Which school does the teacher belongs to"]
data_result = data.filter(["SCHOOL","NAME","RATINGS"])

In [22]:
data_result.head(10)

,SCHOOL,NAME,RATINGS
5,SMEC,ANIL P.M.,3
6,SMEC,ANOJ GIRI,3
19,SMEC,SENTHILKUMAR P,3
23,SMEC,BIBHUTI BHUSAN SAHOO,3
7,SMEC,ARUNA KUMAR BEHURA,2
15,SMEC,NAIJU C D,2
18,SMEC,EDWIN SUDHAGAR P,2
21,SMEC,THIAGARAJAN S,2
2,SMEC,RAHUL SINGH SIKARWAR,1
44,SITE,SWARNA PRIYA R.M,3


In [0]:
data_result.to_json("/content/gdrive/My Drive/Colab Notebooks/review.json",index=False,orient="table")#for backend

In [24]:
drive.mount('/content/gdrive')
model.save("/content/gdrive/My Drive/Colab Notebooks/classifier.h5")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
